In [ ]:
import pandas as pd

df = pd.read_pickle('./data/df_social_data_train.pkl')
df

In [ ]:
# pre-processar e extrair caracteristicas dos dados para construir a tabela atributo-valor
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

df = df.dropna()

In [ ]:
df['features'] = list(model.encode(df['content'].tolist(), show_progress_bar=True))

df

In [ ]:
df[['content','features']].head(10)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
k = 15
X = np.array(df.features.to_list()) # conjunto de dados

kmeans = KMeans(n_clusters=9, random_state=0, n_init="auto").fit(X)
df['cluster'] = kmeans.labels_
df


In [ ]:
import umap

df_sample = df.sample(5000)
X_sample = np.array(df_sample.features.to_list())
Y_sample = np.array(df_sample.cluster.to_list())

features2D = umap.UMAP().fit_transform(X_sample, y=Y_sample)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='poster')

fig, ax = plt.subplots(1, figsize=(7, 5))
plt.scatter(*features2D.T, s=0.1, c=df_sample.cluster.to_list(), cmap='Spectral', alpha=1.0)

In [ ]:
!pip install openai==0.28

In [ ]:
from getpass import getpass
import openai

def llm_local():
  modo_local = input("Deseja usar uma LLM local? (s/n): ").strip().lower()

  if modo_local == 's':
      # Configuração para Ollama local
      openai.api_base = "http://localhost:11434/v1"
      openai.api_key = "ollama"  # Requerido por algumas bibliotecas mesmo que não usado
  else:
      # Configuração para OpenAI
      openai.api_base = "https://api.openai.com/v1"
      openai.api_key = getpass("Digite sua chave de API da OpenAI: ")

In [ ]:
import json
import openai

def llm_task(model, system, prompt):
  response = openai.ChatCompletion.create(
      model=model,
      messages = [
            {
              "role": "system",
              "content": system
            },
            {
              "role": "user",
              "content": prompt
            },
          ]

  )
  s = response['choices'][0]['message']['content'].strip()

  return s

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama
!nohup ollama serve &

In [ ]:
# baixando uma LLM (llama3.1)
!ollama pull llama3.1

In [ ]:
llm_local()

In [ ]:
system = '''
Você é um analista de posts em redes sociais.
A partir de uma lista de posts, você deve:
1) identificar o tema geral dos posts
2) identificar os tópicos mais relevantes nos posts.
3) cada tópico deve ser formado 2 ou 3 palavras
4) sumarizar posts positivos e negativos
5) Apresentar um breve resumo dos posts
A saída da análise deve ser estruturada em JSON.

Exemplo de Estrutura do JSON:

```json
{
  "tema_geral": "texto do tema geral",
  "topicos_relevantes": ["topico A","topico B","topico C"],
  "resumo_positivo": "resumir posts positivos",
  "resumo_negativo": "resumir posts negativos"
}
```

A resposta deve ser em português e APENAS EM JSON.
Não retorne nada além do JSON.
'''





In [ ]:
df.sample(10)

In [ ]:
import json

L = []
temp_dict = {}
for cluster in range(0,9):
  print("Gerando analise do cluster",cluster)
  prompt = ''
  df_temp = df[df.cluster == cluster].sample(10)
  for index,row in df_temp.iterrows():
    prompt += f'#Post: {row.content} ####\n\n\n'
  resposta = llm_task("llama3.1",system,prompt)
  print(resposta)
  json_str = resposta.replace("```json","").replace("```","")
  print(json_str)
  obj = json.loads(json_str)
  print(obj)
  df_temp['tema_geral'] = obj['tema_geral']
  df_temp['topicos_relevantes'] = str(list(obj['topicos_relevantes']))
  df_temp['resumo_positivo'] = str(list(obj['resumo_positivo']))
  df_temp['resumo_negativo'] = str(list(obj['resumo_negativo']))
  L.append(df_temp)
  temp_dict[cluster] = obj
  print("======")


In [ ]:
df_clusters = pd.concat(L)
df_clusters

In [ ]:
df_clusters[['content','tema_geral','topicos_relevantes']]

In [ ]:
for key, value in temp_dict.items():
  print("Cluster: ", key)
  print("Tema Geral: ", value['tema_geral'])
  print("Tópicos Relevantes: ", value['topicos_relevantes'])
  print("Resumo Positivo: ", value['resumo_positivo'])
  print("Resumo Negativo: ", value['resumo_negativo'])
  print("\n")